In [1]:
import requests,time,json,re
from lxml import etree

headers = {
'referer':'https://www.bilibili.com/',
'sec-ch-ua':'" Not A;Brand";v="99", "Chromium";v="100", "Google Chrome";v="100"',
'sec-ch-ua-mobile':'?0',
'sec-ch-ua-platform':'"Windows"',
'sec-fetch-dest':'empty',
'sec-fetch-mode':'cors',
'sec-fetch-site':'same-site',
'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'
}


# 番剧信息
infos_total = []
#评论信息
comments_total = []


'''
番剧信息表字段：

ssid: 番剧id
cartoon: 番剧名
views：播放量
coins：投币数
follow：追番人数
series_follow：系列追番人数
danmakus：弹幕数
likes：点赞量
favorite
favorites：
reply：评论数
share：转发数
cover：封面图
url: 番剧url地址
episodes：集数，如果没有此数据则为-1
count：总计评分人数，如果没有此数据则为-1
score：评分，如果没有此数据则为-1
pub_time: 发布时间
media_tags: 番剧风格/分类
voice_actor: 声优
is_finish: 完结状态 0：未完结 , 1：已完结
'''
def get_cartoonInfos(UrlNum,cartoon):
    resp = requests.get("https://api.bilibili.com/pgc/web/season/stat?season_id={}".format(UrlNum),headers=headers)
    resp.close()
    info = resp.json()['result']
    resp1 = requests.get("https://api.bilibili.com/pgc/view/web/season?season_id={}".format(UrlNum),headers=headers)
    resp1.close()
    print("https://api.bilibili.com/pgc/view/web/season?season_id={}".format(UrlNum))
    info2 = resp1.json()['result']
    media_id = info2['media_id']
    resp1 = requests.get("https://api.bilibili.com/pgc/view/web/season?season_id={}".format(UrlNum),headers=headers)
    resp1.close()
    html = etree.HTML(resp1.text)
    resp1 = requests.get("https://www.bilibili.com/bangumi/media/md{}".format(media_id),headers=headers)
    html = etree.HTML(resp1.text)
    media_tags = html.xpath('//*[@id="app"]/div[1]/div[2]/div/div[2]/div[1]/span[2]')
    media_tag = ""
    for i in media_tags:
        media_tag = i.xpath(".//text()")
    media_tags = ','.join(media_tag)
    actors = re.findall('actors":"(.*?)",',resp1.text)[0]
    actors_ = list(map(lambda x: x.strip(),filter(None,re.split("[：\\\\n]",actors))))[1::2]
    voice_actor = ','.join(actors_)
    
    if info2.get('rating')==None:
        count = -1
        score = -1
    else:
        count = info2['rating']['count']
        score = info2['rating']['score']
    content = {
        'ssid': UrlNum,
        'cartoon': cartoon,
        'views': info['views'],
        'coins': info['coins'],
        'follow': info['follow'],
        'series_follow': info['series_follow'],
        'danmakus': info['danmakus'],
        'likes': info['likes'],
        'favorite': info2['stat']['favorite'],
        'favorites': info2['stat']['favorites'],
        'reply': info2['stat']['reply'],
        'share': info2['stat']['share'],
        'cover': info2['cover'],
        'url': "https://www.bilibili.com/bangumi/play/ss{}".format(UrlNum),
        'episodes': len(info2['episodes']),
        'count': count,
        'is_finish': info2['publish']['is_finish'],
        'pub_time': info2['publish']['pub_time'],
        'media_tags': media_tags,
        'voice_actor': voice_actor,
        'score': score
        
    }
    infos_total.append(content)
    print(content)
    aid = info2['episodes'][0]['aid']
    bvid = info2['episodes'][0]['bvid']
    cid = info2['episodes'][0]['cid']
    return aid,bvid,cid

'''
评论表字段：

mid: 用户id
uname: 用户名
ssid: 番剧id
message: 评论信息
like: 评论点赞数
dt: 评论日期
'''
def get_comments(aid,ssid):
    for page in range(3):
        url = f"https://api.bilibili.com/x/v2/reply?&type=1&pn={page}&oid={aid}"
        r1 = requests.get(url)
        for i in r1.json()['data']['hots']:
            content = {}
            mid = i['member']['mid']
            uname = i['member']['uname']
            message = i['content']['message']
            like = i['like']
            timestamp = i['ctime']
            #转换成localtime
            time_local = time.localtime(timestamp)
            #转换成新的时间格式(2016-05-05 20:28:54)
            dt = time.strftime("%Y-%m-%d %H:%M:%S",time_local)
            content['mid'] = mid
            content['uname'] = uname
            content['ssid'] = ssid
            content['message'] = message
            content['like'] = like
            content['dt'] = dt
            comments_total.append(content)
# 写入Json文件
def saveJson():
    f = open("res/infos_total.json","w",encoding='utf-8')
    f.write(infos_total_Json)
    f.close()
    f = open("res/comments_total.json","w",encoding='utf-8')
    f.write(comments_total_Json)
    f.close()

In [2]:
if __name__ == '__main__':
    r = requests.get("https://www.bilibili.com/v/popular/rank/bangumi",headers=headers)
    html = etree.HTML(r.text)
    ul = html.xpath('//*[@id="app"]/div/div[2]/div[2]/ul')[0]
    for li in ul:
        UrlNum = li.xpath('.//a/@href')[0].split("/")[-1].replace("ss","")
        UrlNum = UrlNum.split("?")[0]
        cartoon = li.xpath('.//a/text()')[0]
        aid,bvid,cid = get_cartoonInfos(UrlNum,cartoon)
        get_comments(aid,UrlNum)
        time.sleep(0.5)
    infos_total_Json = json.dumps(infos_total)
    comments_total_Json = json.dumps(comments_total)
    saveJson()
#     print(infos_total)
#     print(comments_total)


https://api.bilibili.com/pgc/view/web/season?season_id=41410
{'ssid': '41410', 'cartoon': '间谍过家家', 'views': 63330785, 'coins': 622944, 'follow': 6570653, 'series_follow': 6571233, 'danmakus': 280498, 'likes': 1442697, 'favorite': 177344, 'favorites': 6570661, 'reply': 138687, 'share': 166686, 'cover': 'http://i0.hdslb.com/bfs/bangumi/image/a9497ed9b2ad8fd3b77289734769f81bd3948d75.png', 'url': 'https://www.bilibili.com/bangumi/play/ss41410', 'episodes': 2, 'count': -1, 'is_finish': 0, 'pub_time': '2022-04-30 21:00:00', 'media_tags': '漫画改,搞笑,战斗,日常', 'voice_actor': '江口拓也,种崎敦美,早见沙织', 'score': -1}
https://api.bilibili.com/pgc/view/web/season?season_id=41411
{'ssid': '41411', 'cartoon': '辉夜大小姐想让我告白 -究极浪漫-', 'views': 12907110, 'coins': 160460, 'follow': 8900902, 'series_follow': 9049148, 'danmakus': 96949, 'likes': 350751, 'favorite': 27861, 'favorites': 8900902, 'reply': 20891, 'share': 37322, 'cover': 'http://i0.hdslb.com/bfs/bangumi/image/39f7d690deb477004673f40e0fe65c78895c94f4.png', 'url

{'ssid': '41648', 'cartoon': '泡泡', 'views': 6126214, 'coins': 18665, 'follow': 275200, 'series_follow': 275204, 'danmakus': 13502, 'likes': 65139, 'favorite': 9571, 'favorites': 275200, 'reply': 6101, 'share': 20082, 'cover': 'http://i0.hdslb.com/bfs/bangumi/image/640abd023274ca6d94c8d0c460e5a39c1ad8e795.png', 'url': 'https://www.bilibili.com/bangumi/play/ss41648', 'episodes': 1, 'count': 4178, 'is_finish': 1, 'pub_time': '2022-05-01 18:00:00', 'media_tags': '原创,奇幻,热血', 'voice_actor': '志尊淳,Riria,宫野真守,梶裕贵,畠中祐,千本木彩花,井上麻里奈,三木真一郎,广濑爱丽丝', 'score': 7.3}
https://api.bilibili.com/pgc/view/web/season?season_id=41557
{'ssid': '41557', 'cartoon': '这个治疗有点烦', 'views': 4982435, 'coins': 17601, 'follow': 361041, 'series_follow': 361083, 'danmakus': 24303, 'likes': 90010, 'favorite': 5251, 'favorites': 361041, 'reply': 5596, 'share': 8097, 'cover': 'http://i0.hdslb.com/bfs/bangumi/image/91e75030be41d67b9f19b96bb512b0c98ae781bd.png', 'url': 'https://www.bilibili.com/bangumi/play/ss41557', 'episodes': 3

https://api.bilibili.com/pgc/view/web/season?season_id=41558
{'ssid': '41558', 'cartoon': 'RPG不动产', 'views': 4523944, 'coins': 23276, 'follow': 439098, 'series_follow': 439099, 'danmakus': 34699, 'likes': 101852, 'favorite': 6308, 'favorites': 439098, 'reply': 4357, 'share': 3842, 'cover': 'http://i0.hdslb.com/bfs/bangumi/image/92f45d70a0145f3cb292c79d9930db0cdea9c459.png', 'url': 'https://www.bilibili.com/bangumi/play/ss41558', 'episodes': 5, 'count': 1430, 'is_finish': 0, 'pub_time': '2022-04-06 21:30:00', 'media_tags': '漫画改,奇幻,日常,萌系', 'voice_actor': '井上穗乃花,木野日菜,川井田夏海,石见舞菜香', 'score': 9.6}
https://api.bilibili.com/pgc/view/web/season?season_id=5978
{'ssid': '5978', 'cartoon': '博人传 火影忍者新时代', 'views': 286380255, 'coins': 550391, 'follow': 2157297, 'series_follow': 2157296, 'danmakus': 4644655, 'likes': 971958, 'favorite': 22098, 'favorites': 2157297, 'reply': 668093, 'share': 105053, 'cover': 'http://i0.hdslb.com/bfs/bangumi/image/71d54cae830d32a5af776dcc46632146fbdba868.jpg', 'url': '

https://api.bilibili.com/pgc/view/web/season?season_id=2576
{'ssid': '2576', 'cartoon': 'OVERLORD', 'views': 401722495, 'coins': 814805, 'follow': 6166029, 'series_follow': 9713369, 'danmakus': 3829059, 'likes': 974099, 'favorite': 52609, 'favorites': 6166029, 'reply': 190646, 'share': 253747, 'cover': 'http://i0.hdslb.com/bfs/bangumi/b8e72ea69e57e0b7ac85d8b4d366554fd866df1a.jpg', 'url': 'https://www.bilibili.com/bangumi/play/ss2576', 'episodes': 13, 'count': 50341, 'is_finish': 1, 'pub_time': '2015-07-07 00:00:00', 'media_tags': '奇幻,战斗,小说改,架空', 'voice_actor': '日野聪,原由实,上坂堇,加藤英美里,内山夕实,加藤将之,三宅健太,千叶繁,沼仓爱美', 'score': 9.6}
https://api.bilibili.com/pgc/view/web/season?season_id=36170
{'ssid': '36170', 'cartoon': '关于我转生变成史莱姆这档事 第二季', 'views': 372833006, 'coins': 2218554, 'follow': 8952015, 'series_follow': 9973404, 'danmakus': 2585326, 'likes': 4845116, 'favorite': 109283, 'favorites': 8952015, 'reply': 422750, 'share': 126285, 'cover': 'http://i0.hdslb.com/bfs/bangumi/image/fd492888df64bbc3b